In [1]:
$dirEntry = New-Object System.DirectoryServices.DirectoryEntry("LDAP://dc01.gpoabuse.lab", "gpoabuse.lab\john.dee", "johnsPass01")
$searcher = New-Object System.DirectoryServices.DirectorySearcher($dirEntry)
$searcher.Filter = "(&(objectCategory=user)(samAccountName=john.dee))"
$results = $searcher.FindOne()
$results.Properties.distinguishedname
"Groups:"
foreach ($group in $($results.Properties.memberof)) {
    "    $group"
}

CN=John Dee,OU=EnterpriseUsers,DC=gpoabuse,DC=lab
Groups:
    CN=Office Admins,OU=EnterpriseUsers,DC=gpoabuse,DC=lab


In [2]:
$searcher.Filter = "(&(objectCategory=group)(cn=Office Admins))"
$results = $searcher.FindOne()
$groupSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$results.Properties.objectsid[0],0))
$groupSID.Value

S-1-5-21-979066922-3318582297-4097118301-1107


In [3]:
$searcher.Filter = "(objectCategory=groupPolicyContainer)"
$searcher.SecurityMasks = [System.DirectoryServices.SecurityMasks]::Dacl
$groupPolicyObjects = $searcher.FindAll() 

foreach ($gpo in $groupPolicyObjects) {
    # Get the ACL from the binary nTSecurityDescriptor property
    $SecurityDescriptor = New-Object System.DirectoryServices.ActiveDirectorySecurity
    $SecurityDescriptor.SetSecurityDescriptorBinaryForm([Byte[]]$gpo.Properties["nTSecurityDescriptor"][0])
    $gpoACL = $SecurityDescriptor.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])

    # Iterate through the ACLs of each GPO, looking for the targetSID
    foreach ($ace in $gpoACL) {
        if ($ace.IdentityReference.CompareTo($groupSID) -eq 0) {
            $gpoDN = $gpo.Properties.distinguishedname
            $gpoDisplayName = $gpo.Properties.displayname
            $gpoPath = $gpo.Properties.adspath
            Write-Host "$($ace.AccessControlType) -> $($ace.ActiveDirectoryRights) on GPO '$gpoDisplayName'"
        }
    }
}
$dirEntry.Dispose()
$groupPolicyObjects.Dispose()

Allow -> CreateChild, DeleteChild, ReadProperty, WriteProperty, Delete, GenericExecute, WriteDacl, WriteOwner on GPO 'SetWallapper'


In [4]:
$gpoDN

CN={E8F1E2C0-7D8D-439F-91A5-A5D6736EBD70},CN=Policies,CN=System,DC=gpoabuse,DC=lab


In [5]:
$dirEntry = New-Object System.DirectoryServices.DirectoryEntry("LDAP://dc01.gpoabuse.lab", "gpoabuse.lab\john.dee", "johnsPass01")
$searcher = New-Object System.DirectoryServices.DirectorySearcher($dirEntry)
$searcher.Filter = "(objectCategory=organizationalUnit)"
$ouResult = $searcher.FindAll()
ForEach ($ou in $ouResult) {
    if (($ou.Properties.gplink) -match $gpoDN) {
        "'$gpoDisplayName' is linked to $($ou.Properties.name)"
    }
}

$dirEntry.Dispose()
$ouResult.Dispose()

'SetWallapper' is linked to EnterpriseUsers


In [13]:
$gpoEntry = New-Object System.DirectoryServices.DirectoryEntry($gpoPath, "gpoabuse.lab\john.dee", "johnsPass01")
$gpoEntry | select name, distinguishedname, objectCategory, gPCFileSysPath, gPCMachineExtensionNames, gPCUserExtensionNames
$gpoGUID = $gpoEntry.Properties['name']


name                     : {{E8F1E2C0-7D8D-439F-91A5-A5D6736EBD70}}
distinguishedname        : {CN={E8F1E2C0-7D8D-439F-91A5-A5D6736EBD70},CN=Policies,CN=System,DC=gpoa
                           buse,DC=lab}
objectCategory           : {CN=Group-Policy-Container,CN=Schema,CN=Configuration,DC=gpoabuse,DC=lab
                           }
gPCFileSysPath           : {\\gpoabuse.lab\SysVol\gpoabuse.lab\Policies\{E8F1E2C0-7D8D-439F-91A5-A5
                           D6736EBD70}}
gPCMachineExtensionNames : {[{35378EAC-683F-11D2-A89A-00C04FBBCFA2}{D02B1F72-3407-48AE-BA88-E8213C6
                           761F1}]}
gPCUserExtensionNames    : {[{35378EAC-683F-11D2-A89A-00C04FBBCFA2}{D02B1F73-3407-48AE-BA88-E8213C6
                           761F1}]}



In [ ]:
New-SmbMapping -Localpath "X:" -RemotePath "\\dc01.gpoabuse.lab\SysVol\gpoabuse.lab\Policies\" -UserName "gpoabuse\john.dee" -Password "johnsPass01"

New-SmbMapping: 
Line |
   2 |  New-SmbMapping -Localpath "X:" -RemotePath "\\dc01.gpoabuse.lab\SysVo …
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The local device name is already in use. 


Error: The local device name is already in use. 

In [16]:
Set-Location X:\$gpoGUID

In [17]:
ls 


    Directory: X:\{E8F1E2C0-7D8D-439F-91A5-A5D6736EBD70}

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d----            2/9/2025 11:37 PM                Machine
d----            2/9/2025 11:37 PM                User
-a---            2/9/2025 11:37 PM             70 GPO.cmt
-a---            2/9/2025 11:37 PM             64 GPT.INI



In [18]:
(Get-ACL X:\$gpoGUID).Access | where { $_.IdentityReference -match $groupSID }


FileSystemRights  : FullControl
AccessControlType : Allow
IdentityReference : S-1-5-21-979066922-3318582297-4097118301-1107
IsInherited       : False
InheritanceFlags  : ContainerInherit, ObjectInherit
PropagationFlags  : None



In [19]:
Get-Content .\GPT.INI
Write-Host
Get-Content .\GPO.cmt
Write-Host
Get-Content .\User\Registry.pol


[General]
Version=131073
displayName=New Group Policy Object

O f f i c e   a d m i n s   c a n   s e t   t h e   w a l l p a p e r 

PReg   [ C o n t r o l   P a n e l \ C o l o r s   ; B a c k g r o u n d   ;    ;    ; 1 0 0   1 7 5   2 0 0   ] [ C o n t r o l   P a n e l \ D e s k t o p   ; W a l l p a p e r   ;    ;    ;   ] 


In [31]:
$credential = New-Object PSCredential("gpoabuse.lab\jim.duggan", $(ConvertTo-SecureString "jimsPass01" -AsPlainText -Force))
Invoke-Command -ComputerName ws01.gpoabuse.lab -ScriptBlock { whoami /all } -Credential $credential


USER INFORMATION
----------------

User Name           SID                                          
=================== =============================================
gpoabuse\jim.duggan S-1-5-21-979066922-3318582297-4097118301-1106


GROUP INFORMATION
-----------------

Group Name                                      Type             SID                                           Attributes                                                     
=============================================== ================ ============================================= ===============================================================
Everyone                                        Well-known group S-1-1-0                                       Mandatory group, Enabled by default, Enabled group             
BUILTIN\Remote Management Users                 Alias            S-1-5-32-580                                  Mandatory group, Enabled by default, Enabled group             
BUILTIN\Users              

## GPO Overview
Group Policy is a mechanism for deploying computer and user specific configuraion across multiple machines within a network via Active Directory. These configuration items are stored both as a collection of files on a network share, and as an object within the Active Directory itself.

The file share is known as SYSVOL, and specifc folder within this share is known as the Group Policy Template or GPT. The contents of the GPT are replicated across all domain controllers in the domain, so feasibly all DCs should have the copy of the same GPOs. 

The AD container for the GPO objects is called the Group Policy Container or GPC. The GPC contains information such as version of the GPO, whether it is disabled, and 

GPO linking

GPO Scope

GPO Processing is the action of taking a GPO and applying the configuration it specifies. Processing is broken into two parts; the core processing of a GPO firures

Client side extensions (CSEs) are DLLs, that contain the appropriate functionality to actually make the changes to the operating system specified in the GPO. For example, the 

CSEs are loaded into the Group Policy Client Service


| GUID | CSE |
| ---- | --- |
| 00000000-0000-0000-0000-000000000000 | Core GPO Engine |
| 35378EAC-683F-11D2-A89A-00C04FBBCFA2 | Registry Settings |

## Futher Reading

MS docs on how group policy works:
- [Group Policy Architecture](https://learn.microsoft.com/en-us/previous-versions/windows/desktop/policy/group-policy-architecture)

Two brilliant sources for understandng GPO internals: 
- [A Red Teamers Guide to GPOs and OUs](https://posts.specterops.io/a-red-teamers-guide-to-gpos-and-ous-f0d03976a31e)
- [Exploitating Windows Group Policy for Reconnaissance and Attack](https://www.youtube.com/watch?v=eoBeRkpj6TE)

A nice overview of GPO persistence:
- [Sneaky Active Directory Persistence #17: Group Policy](https://adsecurity.org/?p=2716)
